In [1]:
myCollection = "My Name is Darshil and I love Spark"\
.split(" ")

words = spark.sparkContext.parallelize(myCollection, 2)

In [2]:
supplementalData = {"Name":1000, "Darshil":200,
                      "love":-300, "Spark":100}

In [3]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [4]:
suppBroadcast.value

{'Name': 1000, 'Darshil': 200, 'love': -300, 'Spark': 100}

In [5]:
words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
    .sortBy(lambda wordPair: wordPair[1])\
    .collect()

[('love', -300),
 ('My', 0),
 ('is', 0),
 ('and', 0),
 ('I', 0),
 ('Spark', 100),
 ('Darshil', 200),
 ('Name', 1000)]

In [6]:
flights = spark.read.parquet("data/flight-data/parquet/2010-summary.parquet")

In [7]:
flights.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=24),
 Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [22]:
accChina = spark.sparkContext.accumulator(0)

In [23]:
def accChinaFunc(flight_row):
    destination = flight_row["DEST_COUNTRY_NAME"]
    origin = flight_row["ORIGIN_COUNTRY_NAME"]
    
    if destination == "China":
      accChina.add(flight_row["count"])
        
    if origin == "China":
      accChina.add(flight_row["count"])

In [24]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [25]:
accChina.value

953